In [1]:
%set_env TRANSFORMERS_CACHE=/bigstor/zsarwar/models/cache
%set_env CUDA_VISIBLE_DEVICES=0
import time
import argparse
import json
import logging
from pathlib import Path
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import transformers
from transformers import AutoConfig, AutoTokenizer, AutoModelWithLMHead, AutoModelForMaskedLM
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from tqdm import tqdm
from nltk import tokenize
from copy import deepcopy

env: TRANSFORMERS_CACHE=/bigstor/zsarwar/models/cache
env: CUDA_VISIBLE_DEVICES=0


/home/zsarwar/.conda/envs/nlp2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'bert-large-cased'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def load_pretrained(model_name):
    model = AutoModelForMaskedLM.from_pretrained(model_name)
    model.eval()
    tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
    return model, tokenizer

In [4]:
model, tokenizer = load_pretrained(model_name)
model = model.to(device)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# intializing GPT-2
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
gpt_tokenizer.pad_token_id = gpt_tokenizer.eos_token_id
gpt_tokenizer.padding_side = "left"

In [6]:
numpy_file = "/home/zsarwar/NLP/autoprompt/autoprompt/Results/Arrays/correctly_classified_bert_large_cased_autoprompt_format_single_entity_2500_bert-large-cased_gpt_true_adv_token_false_remove_periods_false.npy"
loaded_dict = np.load(numpy_file, allow_pickle=True)
loaded_dict = loaded_dict.item()

In [7]:
all_adv_prompts = []
all_real_labels = []
all_pred_labels = []
all_trigger_start_indices = []

In [8]:
for i in range(len(loaded_dict['results_adversarial']['all_pred_labels_triggers'])):
    temp_prompts = []
    temp_pred_labels = []
    for j in range(len(loaded_dict['results_adversarial']['all_pred_labels_triggers'][0])):
        if(loaded_dict['results_adversarial']['all_pred_labels_triggers'][i][j] == -100):
            continue
        if(loaded_dict['results_adversarial']['all_pred_labels_triggers'][i][j][0] != loaded_dict['results_adversarial']['all_labels_triggers'][i][0]):
            temp_pred_labels.append(loaded_dict['results_adversarial']['all_pred_labels_triggers'][i][j][0])
            temp_prompts.append(loaded_dict['results_adversarial']['all_model_inputs_triggers'][i][j])
    if(len(temp_prompts)!=0):
        all_adv_prompts.append(temp_prompts)
        all_pred_labels.append(temp_pred_labels)
        all_real_labels.append(loaded_dict['results_adversarial']['all_labels_triggers'][i][0])
        all_trigger_start_indices.append(len(loaded_dict['results_baseline']['all_model_inputs'][i]['input_ids'][0])-2)

In [9]:
for i in range(len(all_adv_prompts)):
    for j in range(len(all_adv_prompts[i])):
        if(all_adv_prompts[i][j]!=-100):
            for key in all_adv_prompts[i][j].keys():
                all_adv_prompts[i][j][key] = all_adv_prompts[i][j][key].clone().detach().to(device)

In [10]:
idx=150

In [11]:
print("Adv : ", tokenizer.decode(all_pred_labels[idx][0]))
print("Real : ", tokenizer.decode(all_real_labels[idx]))
print(tokenizer.decode(all_adv_prompts[idx][0]['input_ids'][0]))

Adv :  Canada
Real :  London
[CLS] Arthur Balfour was employed in [MASK]. At the end of the war, he came back to Canada and started a business. [SEP]


In [23]:
all_preds = []
for i in range(len(all_adv_prompts[0:5])):
    sample_preds = []
    for j in range(len(all_adv_prompts[i])):
        trigger_preds = []
        for m in range(all_trigger_start_indices[i], (len(all_adv_prompts[i][j]['input_ids'][0])-1)):
            inputs = deepcopy(all_adv_prompts[i][j])
            inputs['attention_mask'][0][m] = 0
            with torch.no_grad(): 
                logits = model(**inputs).logits
            mask_token_index = (inputs['input_ids'] == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
            predicted_token_id = logits[0, mask_token_index[0]].argmax(axis=-1)
            trigger_preds.append(predicted_token_id)
    sample_preds.append(t_preds)
    all_preds.append(sample_preds)

In [145]:
for i in range(len(all_adv_prompts)):
    for j in range(len(all_adv_prompts[i])):
        if(all_adv_prompts[i][j]!=-100):
            inputs = all_adv_prompts[10][0]
            with torch.no_grad(): 
                logits = model(**inputs).logits
            mask_token_index = (inputs['input_ids'] == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
            predicted_token_id = logits[0, mask_token_index[0]].argmax(axis=-1)
            tokenizer.decode(predicted_token_id)

'Google'